<a href="https://colab.research.google.com/github/ViniciusARZ/GoogleSpreadsheets_DailyUpdate_ETL.ipynb/blob/main/BR_S%C3%A3o_Carlos'_Junior_Enterprises_Nucleos_2021_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Vinícius Antoino Ramos Zecca

*LinkedIn: /viniciusarz*

*Github: /ViniciusARZ*


In [6]:
import sqlite3
import pandas as pd
from IPython.display import display
import sqlalchemy


# Data Cleaning and Processing (w/ Tools)

## Google Spreadsheets

In Google Spreadsheets I cleaned up what it was faster than it would be using SQL:
1. Replace "N/A's with " ", or basically *nulls*
2. Transform some Attribute types to make it easier to read in SQL.
3. Column "Faturamento" was in Text but it actually is a currency, so I used SPLIT() + TRIM() + SUBSTITUTE() + 'Number Formatting' to correct it... Same thing  with others.
4. Select which Columns I'm going to need based on the problem I want to solve and the analysis object. Did a backup in case I wanted to return. 


## SQL - Google Cloud's Big Query

Disclaimers:
1. I didn't use SQLite extension because I already had everything in Big Query
2. The following Query Codes are just to show what I did in SQL and keep a Changelog




Import and Schema Establishment

In [24]:
#'ID:INTEGER,EMPRESA_JUNIOR:STRING,DATA_DE_CADASTRO:DATE,ANO_DE_FEDERACAO:INTEGER, CLUSTER_2020:INTEGER,CLUSTER_2021:INTEGER,CLUSTER_2022:INTEGER,É_ALTO_CRESCIMENTO:BOOLEAN, MES_DO_ALTO_CRESCIMENTO:INTEGER,TEMPO_MEDIO_POR_PROJETO_DIAS:FLOAT,PORC_PROJETOS_COLETADOS_NPS:INTEGER,DATA_DE_ASSINATURA:DATE,MES:INTEGER,PROJETOS:INTEGER,DURACAO_DO_PROJETO_EM_DIAS_UTEIS:INTEGER,FATURAMENTO:FLOAT,ATINGE_ALGUM_ODS:BOOLEAN,NUMERO_DE_ODS_ATINGIDOS:INTEGER,ACAO_COMPARTILHADA:BOOLEAN,NUMERO_DE_EJS_QUE_PARTICIPARAM:INTEGER,NUMERO_DE_MEMBROS_PARTICIPANTES:INTEGER,NPS:INTEGER,DATA_DE_COLETA_DE_NPS:DATE,ESTADO_DO_CLIENTE:STRING,CIDADE_DO_CLIENTE:STRING,SERVICO:STRING,ESTADO:STRING,CIDADE:STRING,UNIVERSIDADE:STRING,CURSOS:STRING,NUMERO_MEMBROS:INTEGER'

# Dataframe reading just to visualize the table
df2021 = pd.read_csv('/content/BJ_NuSC_Mon2021_07 - 2021.csv',',')
display(df2021)


,ID,EMPRESA JÚNIOR,DATA DE CADASTRO,ANO DE FEDERAÇÃO,CLUSTER 2020,CLUSTER 2021,CLUSTER 2022,É DE ALTO CRESCIMENTO?,MÊS DO ALTO CRESCIMENTO,TEMPO MÉDIO POR PROJETO (DIAS),% DE PROJETOS COLETADOS NPS,DATA DE ASSINATURA,MÊS,PROJETOS,DURAÇÃO DO PROJETO (EM DIAS ÚTEIS),FATURAMENTO,ATINGE ALGUM ODS?,Nº DE ODS ATINGIDOS,AÇÃO COMPARTILHADA?,Nº DE EJS QUE PARTICIPARAM,Nº DE MEMBROS PARTICIPANTES,NPS,DATA DE COLETA DE NPS,ESTADO DO CLIENTE,CIDADE DO CLIENTE,SERVIÇO,ESTADO,CIDADE,UNIVERSIDADE,CURSOS,MEMBROS
0,100,EESC jr.,09/09/2016,1992,5.0,5.0,NaN,Não,NaN,38.63,52.0,01/10/2019,10,2,367,"990,000.00",Sim,2,Não,NaN,1.0,NaN,NaN,SP,São Carlos,Desenvolvimento de Aplicativo,SP,São Carlos,UNIVERSIDADE DE SÃO PAULO,"ENGENHARIA AERONÁUTICA, ABI - ENGENHARIA MECÂN...",69
1,100,EESC jr.,09/09/2016,1992,5.0,5.0,NaN,Não,NaN,38.63,52.0,02/03/2020,3,1,240,"600,000.00",Sim,3,Não,NaN,2.0,NaN,NaN,SP,São Carlos,Design de layout,SP,São Carlos,UNIVERSIDADE DE SÃO PAULO,"ENGENHARIA AERONÁUTICA, ABI - ENGENHARIA MECÂN...",69
2,100,EESC jr.,09/09/2016,1992,5.0,5.0,NaN,Não,NaN,38.63,52.0,19/07/2020,7,1,160,"483,000.00",Sim,1,Não,NaN,1.0,NaN,NaN,AM,Manaus,Design Ambiental,SP,São Carlos,UNIVERSIDADE DE SÃO PAULO,"ENGENHARIA AERONÁUTICA, ABI - ENGENHARIA MECÂN...",69
3,100,EESC jr.,09/09/2016,1992,5.0,5.0,NaN,Não,NaN,38.63,52.0,24/07/2020,7,1,116,"400,000.00",Sim,2,Não,NaN,3.0,9.0,30/10/2020,MS,Campo Grande,gestão de resíduos Sólidos,SP,São Carlos,UNIVERSIDADE DE SÃO PAULO,"ENGENHARIA AERONÁUTICA, ABI - ENGENHARIA MECÂN...",69
4,100,EESC jr.,09/09/2016,1992,5.0,5.0,NaN,Não,NaN,38.63,52.0,09/09/2020,9,1,86,"1,393,758.00",Sim,2,Não,NaN,1.0,10.0,20/01/2021,SP,Santa Bárbara D'Oeste,Design de layout,SP,São Carlos,UNIVERSIDADE DE SÃO PAULO,"ENGENHARIA AERONÁUTICA, ABI - ENGENHARIA MECÂN...",69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,2904,EESP Jr.,29/01/2021,2021,NaN,NaN,NaN,Não,NaN,2.41,62.0,19/05/2021,5,1,3,"2,000.00",Sim,2,Não,NaN,1.0,NaN,NaN,SP,São Carlos,Palestras e Workshops,SP,São Carlos,UNIVERSIDADE FEDERAL DE SÃO CARLOS,"EDUCAÇÃO ESPECIAL, EDUCAÇÃO ESPECIAL",14
441,2904,EESP Jr.,29/01/2021,2021,NaN,NaN,NaN,Não,NaN,2.41,62.0,19/05/2021,5,1,3,"2,000.00",Sim,2,Não,NaN,1.0,NaN,NaN,SP,São Carlos,Palestras e Workshops,SP,São Carlos,UNIVERSIDADE FEDERAL DE SÃO CARLOS,"EDUCAÇÃO ESPECIAL, EDUCAÇÃO ESPECIAL",14
442,2904,EESP Jr.,29/01/2021,2021,NaN,NaN,NaN,Não,NaN,2.41,62.0,19/05/2021,5,1,3,"2,000.00",Sim,2,Não,NaN,1.0,NaN,NaN,SP,São Carlos,Palestras e Workshops,SP,São Carlos,UNIVERSIDADE FEDERAL DE SÃO CARLOS,"EDUCAÇÃO ESPECIAL, EDUCAÇÃO ESPECIAL",14
443,2904,EESP Jr.,29/01/2021,2021,NaN,NaN,NaN,Não,NaN,2.41,62.0,19/05/2021,5,1,3,"2,000.00",Sim,2,Não,NaN,1.0,NaN,NaN,SP,São Carlos,Palestras e Workshops,SP,São Carlos,UNIVERSIDADE FEDERAL DE SÃO CARLOS,"EDUCAÇÃO ESPECIAL, EDUCAÇÃO ESPECIAL",14


*The SQL Queries to create Variables are in the Analysis Section below*

# Analysis Phase

In [27]:
# Aggregate data by Junior Enterprise and tell me the Cluster progress rate,
#   so this means that I want to see the proportion between Junior Enterprises that
#   were able to progress from 2020 to 2021 in Cluster classification.
#   QUERY 1
#   SELECT EMPRESA_J__NIOR,
    #   AVG(CLUSTER_2020) AS CLUSTER2020,
    #   AVG(CLUSTER_2021)AS CLUSTER2021,
    #   FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`
    #   GROUP BY EMPRESA_J__NIOR
    #   ORDER BY C2021 DESC
resultsQuery1 = pd.read_csv('results-query1.csv')
display(resultsQuery1)

# What can be seen from there?

# Cluster Evolution
# - In SQL:
#   SELECT EMPRESA_J__NIOR,
    #   AVG(CLUSTER_2020) AS CLUSTER2020,
    #   AVG(CLUSTER_2021)AS CLUSTER2021,
    #     ( SELECT COUNT(ID), 
    #     FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07` 
    #     WHERE CLUSTER_2021 != CLUSTER_2020
    #     ) AS Percent
    #   FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`
    #   GROUP BY EMPRESA_J__NIOR
    #   ORDER BY CLUSTER2021 DESC
# In pandas:
NoJEProgressedCluster = resultsQuery1[resultsQuery1['CLUSTER2020'] != resultsQuery1['CLUSTER2021']]
display(NoJEProgressedCluster.count()/resultsQuery1.count())

# Cluster Proportion
# QUERY 2
#   SELECT CLUSTER_2020, 
    #   count(CLUSTER_2021) as Cluster2021_count,
    #   count(CLUSTER_2021) * 100.0 / (
    #       select count(*) from `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`) 
    #       AS Cluster_percent2021
    #   FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`
    #   group by CLUSTER_2020

,ID,CLUSTER2020,CLUSTER2021
0,100,5.0,5.0
1,163,4.0,5.0
2,421,3.0,5.0
3,137,4.0,5.0
4,400,4.0,4.0
5,302,1.0,4.0
6,131,3.0,4.0
7,142,4.0,4.0
8,168,3.0,4.0
9,1444,1.0,3.0


ID             0.360000
CLUSTER2020    0.333333
CLUSTER2021    0.346939
dtype: float64


If the ID is taken into account, because it is the only value that can't be *null*, this means that only 0.36, or **36%**, of Junior Enterprises were **able to Progress in Cluster Classification**